# 💊 Nosso Agente de Atendimento para Gerenciamento de Medicamentos 🤖

# Este notebook implementa um agente que lembra, registra e gerencia
# a agenda de medicamentos de Pedro no Google Calendar.

In [ ]:
# Importações necessárias
import datetime
import json
import time
import os
import pickle

# Instala as bibliotecas do Google para interagir com a API do Calendar
# Isso precisa ser executado uma vez no início da sessão do Colab.
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

# Importa as classes específicas da API do Google
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Constantes globais do sistema
NOME_ARQUIVO_REGISTRO = "registro_medicamentos_pedro.json" # Arquivo para persistência de dados
TOKEN_FILE = 'token.pickle' # Arquivo para credenciais de autenticação do Google
SCOPES = ['https://www.googleapis.com/auth/calendar'] # Permissões para o Google Calendar

# Dados dos remédios de Pedro
REMEDIOS_PROGRAMADOS = {
    "Sinvastatina": {"horario": "11:00", "dias_da_semana": "todos"},
    "Metiformina": {"horario": "14:00", "dias_da_semana": "todos"},
    "Simeticona": {"horario": "17:00", "dias_da_semana": ["quarta"]}, # Exemplo de remédio em dia específico
    "Tilex": {"horario": "22:00", "dias_da_semana": "todos"}
}

# Mapeamento de números de dias da semana para nomes (útil para lógica e exibição)
# datetime.weekday() retorna 0 para segunda, 1 para terça, ..., 6 para domingo
DIAS_DA_SEMANA_MAP = {
    0: "segunda", 1: "terça", 2: "quarta", 3: "quinta",
    4: "sexta", 5: "sabado", 6: "domingo"
}


In [ ]:
# ... (Seu código existente, incluindo imports e constantes) ...

# --- 2. AUTENTICAÇÃO GOOGLE CALENDAR API ---
def get_google_calendar_service():
    """
    Autentica com a API do Google Calendar e retorna o objeto de serviço.
    Gerencia o token de autenticação (salva/carrega de token.pickle).
    """
    creds = None
    # Verifica se o arquivo token.pickle existe para carregar credenciais anteriores
    if os.path.exists(TOKEN_FILE):
        print(f"Tentando carregar credenciais de '{TOKEN_FILE}'...")
        with open(TOKEN_FILE, 'rb') as token:
            creds = pickle.load(token)

    # Se as credenciais não são válidas ou não existem, inicia o fluxo OAuth 2.0
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print("Credenciais expiradas, tentando renovar...")
            try:
                creds.refresh(Request())
                print("Credenciais renovadas com sucesso.")
            except Exception as e:
                print(f"Falha ao renovar credenciais: {e}. Será necessário autenticar novamente.")
                creds = None # Força nova autenticação
        else:
            # Tenta encontrar credentials.json em vários locais comuns no Colab
            # Ordem de preferência: raiz, sample_data
            credentials_found = False
            for path in ['', 'sample_data/']: # Tenta na raiz e em sample_data
                potential_path = os.path.join(path, 'credentials.json')
                if os.path.exists(potential_path):
                    CREDENTIALS_FILE_PATH = potential_path
                    credentials_found = True
                    print(f"Arquivo 'credentials.json' encontrado em: {CREDENTIALS_FILE_PATH}")
                    break

            if not credentials_found:
                print("\nERRO: O arquivo 'credentials.json' NÃO FOI ENCONTRADO!")
                print("Por favor, faça o upload do arquivo 'credentials.json' (baixado do Google Cloud Console) para a **raiz** ou para a pasta `sample_data` do seu ambiente Colab.")
                print("A integração com o Google Calendar não funcionará sem ele.")
                return None

            try:
                flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE_PATH, SCOPES)
                print("\nPor favor, siga o link para autenticar no Google Calendar:")
                # Tenta usar a porta 0 (dinâmica) e depois 8088 como fallback
                creds = flow.run_local_server(port=0, success_message='Autenticação concluída! Você pode fechar esta aba e retornar ao Colab.')

            except OSError as e:
                print(f"Erro ao tentar usar a porta 0 (dinâmica) para autenticação: {e}")
                print("Tentando a porta 8088 como alternativa...")
                try:
                    creds = flow.run_local_server(port=8088, success_message='Autenticação concluída! Você pode fechar esta aba e retornar ao Colab.')
                except OSError as e_alt:
                    print(f"Erro ao tentar usar a porta 8088: {e_alt}. Não foi possível iniciar o servidor local para autenticação.")
                    print("Por favor, verifique se alguma outra aplicação está usando essas portas ou tente novamente mais tarde.")
                    print("Certifique-se também de que 'http://localhost:8080' E 'http://localhost:8088' estão autorizados em suas credenciais OAuth no Google Cloud Console.")
                    return None # Retorna None se ambas as portas falharem
            except Exception as e:
                print(f"Ocorreu um erro inesperado durante a autenticação: {e}")
                print("Verifique suas credenciais e sua conexão com a internet.")
                return None

        # Salva as credenciais para futuras execuções, APENAS se elas foram obtidas com sucesso
        if creds:
            print(f"Salvando credenciais em '{TOKEN_FILE}'...")
            with open(TOKEN_FILE, 'wb') as token:
                pickle.dump(creds, token)

    # Retorna o serviço do calendário se as credenciais foram obtidas, caso contrário retorna None
    if creds:
        return build('calendar', 'v3', credentials=creds)
    else:
        return None

# ... (Resto do seu código) ...

In [ ]:
def carregar_registro():
    """Carrega o registro de medicamentos de um arquivo JSON."""
    if os.path.exists(NOME_ARQUIVO_REGISTRO):
        with open(NOME_ARQUIVO_REGISTRO, 'r', encoding='utf-8') as f:
            return json.load(f)
    print(f"Arquivo '{NOME_ARQUIVO_REGISTRO}' não encontrado ou vazio. Inicializando novo registro.")
    return inicializar_registro_diario()

def salvar_registro(registro):
    """Salva o registro de medicamentos em um arquivo JSON."""
    with open(NOME_ARQUIVO_REGISTRO, 'w', encoding='utf-8') as f:
        json.dump(registro, f, indent=4, ensure_ascii=False)
    # print("Registro salvo com sucesso.") # Descomente para ver a cada salvamento

def inicializar_registro_diario():
    """
    Inicializa o registro de medicamentos para o dia atual,
    considerando apenas os remédios programados para o dia da semana.
    """
    hoje = datetime.date.today().isoformat()
    registro = {
        "data": hoje,
        "remedios": {}
    }
    dia_semana_hoje = DIAS_DA_SEMANA_MAP[datetime.date.today().weekday()]

    for remedio_nome, info_remedio in REMEDIOS_PROGRAMADOS.items():
        dias_programados = info_remedio["dias_da_semana"]

        # Adiciona o remédio ao registro do dia apenas se for para 'todos' os dias ou para o dia atual
        if dias_programados == "todos" or dia_semana_hoje in dias_programados:
            registro["remedios"][remedio_nome] = {
                "status": "pendente", # 'pendente', 'tomado', 'não tomado'
                "hora_confirmacao": None
            }
    return registro

# Carregar ou inicializar o registro de medicamentos ao iniciar o agente
registro_medicamentos = carregar_registro()

# Verifica se o registro é de um dia anterior e reseta se for para o dia atual
if registro_medicamentos["data"] != datetime.date.today().isoformat():
    print(f"Data do registro anterior ({registro_medicamentos['data']}) diferente da data atual. Reiniciando registro para hoje.")
    registro_medicamentos = inicializar_registro_diario()
    salvar_registro(registro_medicamentos)

print("\n--- INICIALIZAÇÃO DO AGENTE ---")
print(f"Registro de medicamentos carregado/inicializado para o dia: {registro_medicamentos['data']}")
print("\nRemédios configurados para Pedro:")
for remedio, info in REMEDIOS_PROGRAMADOS.items():
    dias = info["dias_da_semana"]
    dias_str = "diariamente" if dias == "todos" else ", ".join([d.capitalize() for d in dias])
    print(f"- {remedio} às {info['horario']} ({dias_str})")
print("-" * 30)


Arquivo 'registro_medicamentos_pedro.json' não encontrado ou vazio. Inicializando novo registro.

--- INICIALIZAÇÃO DO AGENTE ---
Registro de medicamentos carregado/inicializado para o dia: 2025-05-17

Remédios configurados para Pedro:
- Sinvastatina às 11:00 (diariamente)
- Metiformina às 14:00 (diariamente)
- Simeticona às 17:00 (Quarta)
- Tilex às 22:00 (diariamente)
------------------------------


In [ ]:
def mostrar_menu():
    """Exibe as opções que o idoso/cuidador pode escolher."""
    print("\n--- Como posso te ajudar, Pedro? Digite o número da opção: ---")
    print("1. Verificar lembretes de remédios agora") # Para forçar a verificação manual
    print("2. Perguntar sobre meu próximo remédio")
    print("3. Ver o registro de tomada de hoje")
    print("4. Que horas são?")
    print("5. Preciso de ajuda (falar com cuidador)")
    print("6. Gerenciar agenda de remédios no Google Calendar") # Opção de integração
    print("7. Sair do assistente")
    print("-------------------------------------------------------")

def verificar_lembretes_e_interagir(hora_atual_simulada_str):
    """
    Verifica se há algum lembrete de remédio na hora atual simulada e
    interage com o usuário para confirmar a tomada.
    """
    global registro_medicamentos # Permite modificar o registro global

    notificacoes_enviadas = False
    hoje = datetime.date.today()
    dia_semana_hoje = DIAS_DA_SEMANA_MAP[hoje.weekday()]

    for remedio_nome, info_remedio in REMEDIOS_PROGRAMADOS.items():
        horario_programado = info_remedio["horario"]
        dias_programados = info_remedio["dias_da_semana"]

        # 1. Verifica se o remédio é para o dia de hoje
        if dias_programados != "todos" and dia_semana_hoje not in dias_programados:
            continue # Pula este remédio se não for o dia dele

        # 2. Verifica se o remédio está no registro do dia e ainda está pendente
        # (Ele pode não estar no registro se o dia não for o dia da semana dele)
        if remedio_nome not in registro_medicamentos["remedios"] or \
           registro_medicamentos["remedios"][remedio_nome]["status"] != "pendente":
            continue # Pula se não for relevante para o registro de hoje ou já foi tratado

        # 3. Verifica se a hora simulada corresponde ao horário do remédio
        if hora_atual_simulada_str.startswith(horario_programado) and \
           registro_medicamentos["data"] == hoje.isoformat():

            print(f"\n**Lembrete:** Olá, Pedro! É hora de tomar o seu **{remedio_nome}** ({horario_programado}).")
            print("Por favor, tome-o com um copo d'água.")
            resposta = input("Você já tomou o remédio? Digite 'sim' ou 'já tomei' para confirmar: ").lower().strip()

            if resposta in ["sim", "já tomei"]:
                print(f"Ótimo! Fico feliz que você se cuidou e tomou o {remedio_nome}!")
                registro_medicamentos["remedios"][remedio_nome]["status"] = "tomado"
                registro_medicamentos["remedios"][remedio_nome]["hora_confirmacao"] = datetime.datetime.now().strftime("%H:%M:%S")
            else:
                print(f"Entendido. Por favor, lembre-se de tomar o {remedio_nome} em breve.")
                registro_medicamentos["remedios"][remedio_nome]["status"] = "não tomado"
                registro_medicamentos["remedios"][remedio_nome]["hora_confirmacao"] = datetime.datetime.now().strftime("%H:%M:%S")

            salvar_registro(registro_medicamentos) # Salva a cada atualização
            notificacoes_enviadas = True
    return notificacoes_enviadas

def encontrar_proximo_remedio(hora_atual_simulada_str):
    """
    Encontra o próximo remédio a ser tomado, considerando o registro do dia,
    os dias da semana programados e a hora atual.
    """
    agora_time = datetime.datetime.strptime(hora_atual_simulada_str, "%H:%M:%S").time()
    hoje = datetime.date.today()
    dia_semana_hoje = DIAS_DA_SEMANA_MAP[hoje.weekday()]

    proximo_horario_obj = None
    proximo_nome = None

    # Filtra os remédios que são para hoje e ainda estão no registro como pendentes
    remedios_pendentes_hoje = []
    for remedio_nome, info_remedio in REMEDIOS_PROGRAMADOS.items():
        dias_programados = info_remedio["dias_da_semana"]
        if (dias_programados == "todos" or dia_semana_hoje in dias_programados) and \
           remedio_nome in registro_medicamentos["remedios"] and \
           registro_medicamentos["remedios"][remedio_nome]["status"] == "pendente":
            remedios_pendentes_hoje.append((remedio_nome, info_remedio["horario"]))

    # Ordena os remédios pendentes pelo horário
    remedios_pendentes_hoje_ordenados = sorted(
        remedios_pendentes_hoje,
        key=lambda item: datetime.datetime.strptime(item[1], "%H:%M").time()
    )

    # Procura o próximo remédio que ainda não foi tomado e cujo horário ainda não passou
    for remedio_nome, horario_str in remedios_pendentes_hoje_ordenados:
        horario_remedio_obj = datetime.datetime.strptime(horario_str, "%H:%M").time()

        if horario_remedio_obj >= agora_time: # '>=' para incluir o remédio da hora atual se ainda pendente
            proximo_horario_obj = horario_remedio_obj
            proximo_nome = remedio_nome
            break # Encontrou o próximo para hoje

    # Se não encontrou nenhum remédio pendente para as próximas horas de hoje,
    # verifica se tem algum pendente de hoje que já deveria ter sido tomado (atrasado)
    if not proximo_nome:
        for remedio_nome, horario_str in remedios_pendentes_hoje_ordenados:
            horario_remedio_obj = datetime.datetime.strptime(horario_str, "%H:%M").time()
            if horario_remedio_obj < agora_time:
                proximo_horario_obj = horario_remedio_obj
                proximo_nome = remedio_nome
                print(f"\n**ATENÇÃO:** O remédio **{remedio_nome}** das {horario_str} está pendente e o horário já passou.")
                break # Encontrou o primeiro remédio atrasado

    if proximo_nome:
        print(f"\nSeu próximo remédio é o **{proximo_nome}** às **{proximo_horario_obj.strftime('%H:%M')}**.")
        if registro_medicamentos["remedios"][proximo_nome]["status"] == "pendente" and proximo_horario_obj < agora_time:
             print("Por favor, tome-o o mais rápido possível.")
    else:
        print("\nParece que você já tomou todos os remédios programados para hoje, ou não há mais remédios pendentes para o dia.")
        print("Para ver o registro completo do dia, escolha a opção '3' no menu.")

def ver_registro_completo_do_dia():
    """Mostra o status de todos os remédios do dia."""
    print("\n--- Registro de Remédios de Hoje para Pedro ---")
    print(f"Data: {registro_medicamentos['data']}")
    if not registro_medicamentos["remedios"]:
        print("Nenhum remédio programado para o dia de hoje.")
    else:
        for remedio_nome, info in registro_medicamentos["remedios"].items():
            # Acessa o horário programado a partir de REMEDIOS_PROGRAMADOS para consistência
            horario_programado = REMEDIOS_PROGRAMADOS.get(remedio_nome, {}).get("horario", "N/A")

            status = info['status']
            hora_confirmacao = info['hora_confirmacao'] if info['hora_confirmacao'] else "Não confirmado"
            print(f"- {remedio_nome} ({horario_programado}): {status.capitalize()} (Confirmado às: {hora_confirmacao})")
    print("------------------------------------")

def informar_cuidador():
    """Simula o envio de uma mensagem para o cuidador."""
    print("\n**AVISO IMPORTANTE:**")
    print("Sua solicitação de ajuda foi enviada para o cuidador responsável.")
    print("Por favor, aguarde, ele(a) entrará em contato em breve.")
    print("(Em um sistema real, isso enviaria uma mensagem SMS, WhatsApp ou e-mail para o cuidador.)")


In [ ]:
def criar_ou_atualizar_evento_google_calendar(service, remedio_nome, horario, dias_da_semana):
    """
    Cria ou atualiza um evento recorrente no Google Calendar para o remédio especificado.
    """
    summary = f"💊 Lembrete Remédio: {remedio_nome} (Pedro)"
    description = f"Hora de tomar o seu {remedio_nome}, Pedro! Por favor, não se esqueça."

    hoje = datetime.date.today()
    data_hora_inicio_str = f"{hoje.isoformat()}T{horario}:00"

    # Eventos de 0 minutos de duração são bons para lembretes pontuais
    data_hora_fim_str = f"{hoje.isoformat()}T{horario}:00"

    # Constrói a regra de recorrência (RRULE)
    rrule_str = "FREQ=DAILY" # Padrão para todos os dias
    if dias_da_semana != "todos":
        # Converte para o formato de dias da semana do Google Calendar (MO, TU, WE, etc.)
        dias_gc = [d.upper()[:2] for d in dias_da_semana]
        rrule_str = f"FREQ=WEEKLY;BYDAY={','.join(dias_gc)}"

    event = {
        'summary': summary,
        'description': description,
        'start': {
            'dateTime': data_hora_inicio_str,
            'timeZone': 'America/Sao_Paulo', # IMPORTANTE: Ajuste para o fuso horário correto do idoso
        },
        'end': {
            'dateTime': data_hora_fim_str,
            'timeZone': 'America/Sao_Paulo',
        },
        'recurrence': [
            f'RRULE:{rrule_str}' # Define a repetição do evento
        ],
        'reminders': {
            'useDefault': False, # Não usa as configurações de lembrete padrão do usuário
            'overrides': [
                {'method': 'popup', 'minutes': 0},  # Notificação pop-up na hora
                {'method': 'popup', 'minutes': 10}, # Notificação pop-up 10 minutos antes
            ],
        },
    }

    try:
        # Tenta encontrar um evento existente com o mesmo resumo (nome)
        # Isso é uma simplificação. Em um sistema robusto, você guardaria os IDs dos eventos do GC.
        events_result = service.events().list(calendarId='primary', q=summary,
                                              timeMin=datetime.datetime.utcnow().isoformat() + 'Z',
                                              maxResults=1).execute()
        events = events_result.get('items', [])

        if events:
            # Se encontrar, atualiza o evento existente
            event_id = events[0]['id']
            updated_event = service.events().update(calendarId='primary', eventId=event_id, body=event).execute()
            print(f"🗓️ Evento '{remedio_nome}' ATUALIZADO no Google Calendar.")
        else:
            # Se não encontrar, cria um novo evento
            event = service.events().insert(calendarId='primary', body=event).execute()
            print(f"🗓️ Evento '{remedio_nome}' CRIADO no Google Calendar: {event.get('htmlLink')}")

    except HttpError as error:
        print(f"❌ Erro ao criar/atualizar evento no Google Calendar para {remedio_nome}: {error}")
    except Exception as e:
        print(f"❌ Ocorreu um erro inesperado ao lidar com o calendário para {remedio_nome}: {e}")

def gerenciar_agenda_google_calendar():
    """
    Orquestra a criação/atualização de todos os eventos de remédios no Google Calendar.
    """
    if not calendar_service:
        print("\n❌ Não foi possível acessar o Google Calendar. Por favor, verifique a conexão e o arquivo 'credentials.json'.")
        return

    print("\n--- INICIANDO GERENCIAMENTO DA AGENDA NO GOOGLE CALENDAR ---")
    for remedio_nome, info_remedio in REMEDIOS_PROGRAMADOS.items():
        horario = info_remedio["horario"]
        dias_da_semana = info_remedio["dias_da_semana"]
        criar_ou_atualizar_evento_google_calendar(calendar_service, remedio_nome, horario, dias_da_semana)
    print("--- GERENCIAMENTO DA AGENDA CONCLUÍDO. ---")

# Executa o gerenciamento da agenda no Google Calendar logo na inicialização
# Isso garante que a agenda esteja sempre atualizada quando o agente "liga".
gerenciar_agenda_google_calendar()


❌ Não foi possível acessar o Google Calendar. Por favor, verifique a conexão e o arquivo 'credentials.json'.


In [ ]:
print("\n--- AGENTE DE ATENDIMENTO ONLINE PARA PEDRO ---")
print("Para sair a qualquer momento, digite '7' no menu.")

# ATENÇÃO: Mude esta variável para simular diferentes momentos do dia
# Exemplo: "11:00:00" para Sinvastatina, "14:00:00" para Metiformina, etc.
# Para a Simeticona, lembre-se que só funciona se a data atual for quarta-feira.
hora_atual_para_teste = "11:00:00" # <<< DEFINA A HORA SIMULADA PARA TESTES AQUI

agente_ativo = True
while agente_ativo:
    print(f"\n--- HORA SIMULADA: {hora_atual_para_teste} ---")
    hoje = datetime.date.today()
    dia_semana_simulado = DIAS_DA_SEMANA_MAP[hoje.weekday()]
    print(f"--- DIA DA SEMANA REAL: {dia_semana_simulado.capitalize()} ---")

    # Primeiro, verifica e interage com os lembretes na hora simulada atual
    verificar_lembretes_e_interagir(hora_atual_para_teste)

    # Depois dos lembretes (ou se não houver um), mostra o menu para interação
    mostrar_menu()
    escolha = input("Sua escolha (1-7): ").strip()

    if escolha == '1':
        hora_para_verificar = input("Qual hora você quer verificar para lembretes? (Ex: HH:MM:SS) ").strip()
        verificar_lembretes_e_interagir(hora_para_verificar)
    elif escolha == '2':
        encontrar_proximo_remedio(hora_atual_para_teste)
    elif escolha == '3':
        ver_registro_completo_do_dia()
    elif escolha == '4':
        print(f"\nAgora são (hora do seu computador): {datetime.datetime.now().strftime('%H:%M:%S')}")
        print(f"A hora simulada para o agente é: {hora_atual_para_teste}")
    elif escolha == '5':
        informar_cuidador()
    elif escolha == '6':
        gerenciar_agenda_google_calendar() # Aciona a função de integração com o Calendar
    elif escolha == '7':
        print("\nObrigado por usar o Assistente de Remédios, Pedro! Tenha um bom dia.")
        agente_ativo = False
    else:
        print("Opção inválida. Por favor, digite um número de 1 a 7.")

    # Pequena pausa para tornar a leitura mais confortável entre as interações
    time.sleep(1)

print("\n--- AGENTE DE ATENDIMENTO ENCERRADO. ---")


--- AGENTE DE ATENDIMENTO ONLINE PARA PEDRO ---
Para sair a qualquer momento, digite '7' no menu.

--- HORA SIMULADA: 11:00:00 ---
--- DIA DA SEMANA REAL: Sabado ---

**Lembrete:** Olá, Pedro! É hora de tomar o seu **Sinvastatina** (11:00).
Por favor, tome-o com um copo d'água.
Você já tomou o remédio? Digite 'sim' ou 'já tomei' para confirmar: SIM
Ótimo! Fico feliz que você se cuidou e tomou o Sinvastatina!

--- Como posso te ajudar, Pedro? Digite o número da opção: ---
1. Verificar lembretes de remédios agora
2. Perguntar sobre meu próximo remédio
3. Ver o registro de tomada de hoje
4. Que horas são?
5. Preciso de ajuda (falar com cuidador)
6. Gerenciar agenda de remédios no Google Calendar
7. Sair do assistente
-------------------------------------------------------
Sua escolha (1-7): 7

Obrigado por usar o Assistente de Remédios, Pedro! Tenha um bom dia.

--- AGENTE DE ATENDIMENTO ENCERRADO. ---
